Install required libraries

In [1]:
!pip install tensorflow pandas numpy scikit-learn spacy faiss-cpu sentence-transformers joblib

In [2]:
!pip install tensorflow pandas numpy scikit-learn spacy faiss-cpu sentence-transformers joblib
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 3.3 MB/s eta 0:00:04
     ---- ----------------------------------- 1.6/12.8 MB 3.7 MB/s eta 0:00:04
     -------- ------------------------------- 2.6/12.8 MB 4.1 MB/s eta 0:00:03
     ----------- ---------------------------- 3.7/12.8 MB 4.4 MB/s eta 0:00:03
     --------------- ------------------------ 5.0/12.8 MB 4.7 MB/s eta 0:00:02
     ------------------- -------------------- 6.3/12.8 MB 5.0 MB/s eta 0:00:02
     ----------------------- ---------------- 7.6/12.8 MB 5.2 MB/s eta 0:00:02
     --------------------------- ------------ 8.9/12.8 MB 5.4 MB/s eta 0:00:01
     ------------------------------- -------- 10.2/12.8 MB 5.4 MB/s eta 0:00:01
     ------------------------------------ --- 11.8/12.8 MB 5.6 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 5.6 M

In [3]:
!pip install tf-keras

In [4]:
import pandas as pd
import numpy as np
import joblib
import faiss
import pickle
import spacy
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
import warnings
warnings.filterwarnings("ignore")

C:\Users\Reddy Sekhar\AppData\Local\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Load dataset

In [5]:
df = pd.read_csv("BankFAQs.csv")
df.head()

,Question,Answer,Class
0,What are the documents required for opening a ...,Following documents are required to open a Cur...,accounts
1,Can I transfer my Current Account from one bra...,"Yes, Current Accounts can be transferred from ...",accounts
2,My present status is NRI. What extra documents...,NRI/PIO can open the proprietorship/partnershi...,accounts
3,What are the documents required for opening a ...,Following documents are required for opening a...,accounts
4,What documents are required to change the addr...,Following documents are required to change the...,accounts


In [6]:
#Prepare texts and labels
texts = df["Question"].astype(str).tolist()
answers = df["Answer"].astype(str).tolist()
labels = df["Class"].astype(str).tolist()

In [7]:
# Label-encode intents
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
joblib.dump(label_encoder, "label_encoder.pkl")
len(label_encoder.classes_), label_encoder.classes_[:10]

(7,
 array(['accounts', 'cards', 'fundstransfer', 'insurance', 'investments',
        'loans', 'security'], dtype='<U13'))

In [8]:
# Tokenization & Padding (TensorFlow/Keras)
vocab_size = 12000
max_len = 40
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
joblib.dump(tokenizer, "tokenizer.pkl")
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_len, padding="post")

In [9]:
# Train–test split
X_train, X_test, y_train, y_test = train_test_split(padded, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
# Build TensorFlow/Keras model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_len))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dense(len(np.unique(y)), activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Train model
history = model.fit(X_train,y_train,validation_split=0.2,epochs=6,batch_size=32,verbose=1)

Epoch 1/6
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - accuracy: 0.2875 - loss: 1.7293 - val_accuracy: 0.4507 - val_loss: 1.6007
Epoch 2/6
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.5326 - loss: 1.3158 - val_accuracy: 0.6937 - val_loss: 1.0163
Epoch 3/6
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.7522 - loss: 0.7431 - val_accuracy: 0.7852 - val_loss: 0.6700
Epoch 4/6
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.8739 - loss: 0.3943 - val_accuracy: 0.8380 - val_loss: 0.5060
Epoch 5/6
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.9224 - loss: 0.2574 - val_accuracy: 0.8521 - val_loss: 0.5063
Epoch 6/6
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.9515 - loss: 0.1770 - val_accuracy: 0.8592 - val_loss: 0.6237


In [12]:
# Evaluate intent classifier
y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test,y_pred,target_names=label_encoder.classes_))

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
               precision    recall  f1-score   support

     accounts       0.95      0.86      0.90        63
        cards       0.78      0.96      0.86        81
fundstransfer       0.00      0.00      0.00         3
    insurance       0.94      0.94      0.94        94
  investments       0.48      0.43      0.45        28
        loans       0.96      0.92      0.94        75
     security       0.71      0.45      0.56        11

     accuracy                           0.86       355
    macro avg       0.69      0.65      0.66       355
 weighted avg       0.86      0.86      0.86       355



In [13]:
# Save intent model
model.save("intent_keras.h5")

In [14]:
import tensorflow_hub as hub
import numpy as np
import faiss
import pickle

# 1) Take answers from your dataset
answers = df["Answer"].astype(str).tolist()

# 2) Load Universal Sentence Encoder
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# 3) Embed answers
embeddings = use_model(answers).numpy()

# 4) Build FAISS index with correct dimension (512)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype("float32"))

# 5) Save new index and answers
faiss.write_index(index, "faq_index.faiss")

with open("faq_answers.pkl", "wb") as f:
    pickle.dump(answers, f)

print("FAISS index rebuilt successfully using USE embeddings.")


FAISS index rebuilt successfully using USE embeddings.


In [15]:
# Load components for chatbot
nlp = spacy.load("en_core_web_sm")
intent_model = tf.keras.models.load_model("intent_keras.h5")
tokenizer = joblib.load("tokenizer.pkl")
label_encoder = joblib.load("label_encoder.pkl")
faq_index = faiss.read_index("faq_index.faiss")
answers = pickle.load(open("faq_answers.pkl", "rb"))

In [16]:
# Define chatbot functions
def predict_intent(text):
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=max_len, padding="post")
    pred = np.argmax(intent_model.predict(pad), axis=1)[0]
    return label_encoder.inverse_transform([pred])[0]

def get_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

def rag_answer(text):
    query_emb = rag_model.encode([text])
    distances, indices = faq_index.search(np.array(query_emb).astype("float32"), k=1)
    return answers[indices[0][0]]

def chatbot_reply(user_input):
    intent = predict_intent(user_input)
    ents = get_entities(user_input)
    ans = rag_answer(user_input)

    print("Detected Intent:", intent)
    print("Entities:", ents)
    print("Response:", ans)

In [17]:
import tensorflow_hub as hub
import faiss
import pickle
import numpy as np

# Load Universal Sentence Encoder
rag_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Load FAISS index and answers saved earlier
faq_index = faiss.read_index("faq_index.faiss")

with open("faq_answers.pkl", "rb") as f:
    answers = pickle.load(f)

def rag_answer(text):
    # embed query using USE
    query_emb = rag_model([text]).numpy()
    
    # search in FAISS
    distances, indices = faq_index.search(query_emb.astype("float32"), k=1)
    
    # return top answer
    return answers[indices[0][0]]


In [18]:
# Test chatbot
chatbot_reply("How to open a current account?")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 815ms/step
Detected Intent: accounts
Entities: []
Response: The initial deposit amount to open a Flexi Current account is Rs.75,000.


In [19]:
chatbot_reply("Documents required for KYC?")
chatbot_reply("How can I transfer my account to another branch?")
chatbot_reply("What are branch timings?")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Detected Intent: investments
Entities: [('KYC', 'ORG')]
Response: There no need for any documents to be submitted.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Detected Intent: accounts
Entities: []
Response: Following is the process to be followed to transfer your account from one branch to another: An account transfer form, signed by all applicants, given at any of our branches. We recommend that you submit the form at the branch that you want your account transferred to. There will be no change in your account number. You can continue to use your existing chequebook, debit card etc
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Detected Intent: investments
Entities: []
Response: Yes, Current Accounts can be transferred from one branch to another. However, there are certain restrictions. Please visit your nearest branch for details.
